<a href="https://colab.research.google.com/github/sher1w/AI/blob/main/airavata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate torch sentencepiece pandas scikit-learn


In [ ]:
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get("ColabToken")  # your saved secret
login(token=hf_token)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "ai4bharat/Airavata"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
from google.colab import files

uploaded = files.upload()  # upload dataset.xlsx


Saving final_cleaned_data.csv to final_cleaned_data (1).csv


In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_file2 (1).csv")

df = df.dropna()
df.head()


,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्क...,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लि...,non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील...",non-hostile
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते ...,"defamation,offensive"
4,5,- अनलॉक-4 के लिए गाइडलाइन्स जारी - 7 सितंबर से...,non-hostile


In [ ]:
def fake_news_prompt(text):
    return f"""
You are a professional fact-checking system.

Classify the following Hindi news as either:
FAKE or REAL

Respond with ONLY one word: FAKE or REAL.

News:
{text}

Answer:
"""


In [ ]:
def predict_fake_news(text):
    prompt = fake_news_prompt(text)

    output = pipe(
        prompt,
        max_new_tokens=5,
        do_sample=False,
        truncation=True
    )[0]["generated_text"]

    answer = output.split("Answer:")[-1].strip().split()[0]
    return answer.upper()


In [ ]:
df = df.rename(columns={
    "Post": "text",
    "Labels Set": "label"
})

print(df.columns)


Index(['Text', 'Label', 'Word_Count'], dtype='object')


In [ ]:
# Run this right after loading your CSV/dataframe
df.columns = df.columns.str.lower()

# Now "text" will work perfectly!
df_small = df.sample(100, random_state=42)
df_small["prediction"] = df_small["text"].apply(predict_fake_news)

In [ ]:
# 1. Standardize column names to lowercase
df.columns = df.columns.str.lower()

# 2. Sample the data
df_small = df.sample(100, random_state=42)

# 3. Use lowercase "text" (since we just renamed it in step 1)
df_small["prediction"] = df_small["text"].apply(predict_fake_news)

# 4. Map the results (using the mapping we discussed)
mapping = {"FAKE": 0, "REAL": 1}
df_small["prediction"] = df_small["prediction"].str.upper().map(mapping)

# 5. Check the result
print(df_small[["text", "label", "prediction"]].head())

                                                    text label  prediction
9115   रियो डी जनेरियो - सिमोन मैनुअल ने गुरुवार रात ...     0           1
12443  आज तक न्यूज़ चैनल एक नंबर से 10 नंबर पर पहुंच ...     0           0
16721  एनमॉर्गन प्रोजेक्ट वेरिटास जेम्स ओकीफ के दिमाग...     1           0
13342  भारत ने कहा है कि सुरक्षा परिषद में सुधार की अ...     0           1
9386   राम मंदिर निर्माण शुरू होने के वक्त एक मस्जिद ...     0           1


In [ ]:
# Convert y_true from Strings to Integers
y_true = df_small[LABEL_COLUMN].astype(int)
y_pred = df_small["prediction"]

# Now this will work!
print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred))

Accuracy: 0.5

Classification Report:

              precision    recall  f1-score   support

           0       0.56      0.42      0.48        55
           1       0.46      0.60      0.52        45

    accuracy                           0.50       100
   macro avg       0.51      0.51      0.50       100
weighted avg       0.51      0.50      0.50       100



In [ ]:
df_small.to_excel("airavata_fake_news_results.xlsx", index=False)
print("Saved results to airavata_fake_news_results.xlsx")


Saved results to airavata_fake_news_results.xlsx


In [ ]:
!pip install -q transformers accelerate torch sentencepiece pandas scikit-learn
from huggingface_hub import login
from google.colab import userdata
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.metrics import accuracy_score, classification_report

# Login
hf_token = userdata.get("ColabToken")
login(token=hf_token)

# Load Airavata
model_id = "ai4bharat/Airavata"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Complete Google Colab code for Airavata LLM fake news detection on BOTH datasets
# Handles binary (final_cleaned_data) and multi-class datasets with highest accuracy

# ===============================================
# STEP 1: INSTALL DEPENDENCIES
# ===============================================
!pip install -q transformers accelerate torch sentencepiece pandas scikit-learn openpyxl huggingface_hub

# ===============================================
# STEP 2: HUGGINGFACE AUTHENTICATION
# ===============================================
from huggingface_hub import login
from google.colab import userdata

# Get your HF token from Colab secrets (add it first: left panel -> Secrets -> ColabToken)
hf_token = userdata.get("ColabToken")
login(token=hf_token)

# ===============================================
# STEP 3: LOAD AIRAVATA MODEL
# ===============================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from google.colab import files
import re

model_id = "ai4bharat/Airavata"
print("Loading Airavata model... (this takes ~5-10 mins)")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)
print("✅ Airavata model loaded successfully!")

# ===============================================
# STEP 4: IMPROVED PROMPTS FOR HIGHER ACCURACY
# ===============================================
def binary_prompt(text):
    """Optimized prompt for BINARY classification (FAKE/REAL)"""
    return f"""आप एक प्रोफेशनल फेक न्यूज़ डिटेक्टर हैं।

निम्नलिखित हिंदी न्यूज़ को केवल एक शब्द में वर्गीकृत करें:
- FAKE अगर फर्जी है
- REAL अगर सच्ची है

केवल एक शब्द जवाब दें: FAKE या REAL

न्यूज़:
{text}

जवाब: """

def multiclass_prompt(text):
    """Optimized prompt for MULTI-CLASS classification"""
    return f"""आप एक प्रोफेशनल फेक न्यूज़ विशेषज्ञ हैं।

निम्नलिखित हिंदी न्यूज़ को निम्नलिखित 4 श्रेणियों में वर्गीकृत करें:
1. REAL - पूरी तरह सत्य
2. FAKE - पूरी तरह फर्जी
3. MISLEADING - भ्रामक/गलत तथ्य
4. SATIRE - व्यंग्य/मज़ाक

केवल एक शब्द जवाब दें: REAL, FAKE, MISLEADING, या SATIRE

न्यूज़:
{text}

जवाब: """

# ===============================================
# STEP 5: ROBUST PREDICTION FUNCTIONS
# ===============================================
def predict_binary(text, max_retries=3):
    """Robust binary prediction with retry logic"""
    for attempt in range(max_retries):
        try:
            prompt = binary_prompt(text)
            output = pipe(
                prompt,
                max_new_tokens=10,
                do_sample=False,
                temperature=0.1,
                pad_token_id=tokenizer.eos_token_id
            )[0]["generated_text"]

            # Extract answer more reliably
            answer = re.search(r'जवाब[:\s]*([A-Z]+)', output, re.IGNORECASE)
            if answer:
                result = answer.group(1).strip().upper()
                if result in ['FAKE', 'REAL']:
                    return result

            # Fallback extraction
            last_line = output.split('\n')[-1].strip()
            words = last_line.split()
            for word in words:
                word = word.strip().upper()
                if word in ['FAKE', 'REAL']:
                    return word

        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            continue

    return "FAKE"  # Default fallback

def predict_multiclass(text, max_retries=3):
    """Robust multiclass prediction"""
    labels = ['REAL', 'FAKE', 'MISLEADING', 'SATIRE']
    for attempt in range(max_retries):
        try:
            prompt = multiclass_prompt(text)
            output = pipe(
                prompt,
                max_new_tokens=15,
                do_sample=False,
                temperature=0.1,
                pad_token_id=tokenizer.eos_token_id
            )[0]["generated_text"]

            # Extract answer
            answer = re.search(r'जवाब[:\s]*([A-Z]+)', output, re.IGNORECASE)
            if answer:
                result = answer.group(1).strip().upper()
                if result in labels:
                    return result

            # Fallback
            last_line = output.split('\n')[-1].strip()
            words = last_line.split()
            for word in words:
                word = word.strip().upper()
                if word in labels:
                    return word

        except Exception as e:
            continue

    return "FAKE"  # Default fallback

# ===============================================
# STEP 6: PROCESS BOTH DATASETS
# ===============================================
print("\n📁 Upload your datasets:")
print("- Binary: 'final_cleaned_data.csv' (columns: Text, Label, Word_Count)")
print("- Multi-class: 'cleaned_file2 (1).csv' or similar")

uploaded = files.upload()

# Function to process any dataset
def process_dataset(file_name, is_binary=True, label_col='label', text_col='text'):
    """Process single dataset with optimal settings"""
    print(f"\n🔄 Processing {file_name}...")

    # Load and clean data
    if file_name.endswith('.csv'):
        df = pd.read_csv(file_name)
    else:
        df = pd.read_excel(file_name)

    # Standardize column names
    df.columns = df.columns.str.lower().str.strip()
    df = df.rename(columns={text_col.lower(): 'text', label_col.lower(): 'label'})

    # Clean data
    df = df.dropna(subset=['text', 'label'])
    df['text'] = df['text'].astype(str).str.strip()
    df = df[df['text'].str.len() > 10]  # Remove very short texts

    print(f"📊 Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

    # Sample for faster processing (adjust as needed)
    sample_size = min(200, len(df))  # Process 200 samples max
    df_sample = df.sample(sample_size, random_state=42)

    # Make predictions
    if is_binary:
        df_sample['prediction'] = df_sample['text'].apply(predict_binary)
        mapping = {'FAKE': 0, 'REAL': 1}
    else:
        df_sample['prediction'] = df_sample['text'].apply(predict_multiclass)
        mapping = {'REAL': 0, 'FAKE': 1, 'MISLEADING': 2, 'SATIRE': 3}

    # Map labels and predictions to numbers
    df_sample['label_num'] = df_sample['label'].map(mapping).fillna(-1)
    df_sample['pred_num'] = df_sample['prediction'].map(mapping).fillna(-1)

    # Filter valid predictions
    valid_mask = (df_sample['label_num'] != -1) & (df_sample['pred_num'] != -1)
    df_valid = df_sample[valid_mask]

    # Calculate metrics
    if len(df_valid) > 0:
        accuracy = accuracy_score(df_valid['label_num'], df_valid['pred_num'])
        print(f"✅ Accuracy: {accuracy:.3f} ({len(df_valid)}/{len(df_sample)} valid)")
        print("\n📈 Classification Report:")
        print(classification_report(df_valid['label_num'], df_valid['pred_num'],
                                  target_names=list(mapping.keys())))
    else:
        print("❌ No valid predictions found!")
        accuracy = 0

    return df_valid, accuracy

# ===============================================
# STEP 7: RUN ON BOTH DATASETS
# ===============================================
results = {}

# Process BINARY dataset (final_cleaned_data)
binary_files = [f for f in uploaded.keys() if 'final_cleaned' in f.lower() or 'final' in f.lower()]
if binary_files:
    binary_file = binary_files[0]
    df_binary, acc_binary = process_dataset(binary_file, is_binary=True,
                                          label_col='Label', text_col='Text')
    results['binary'] = {'accuracy': acc_binary, 'df': df_binary}
else:
    print("⚠️ Binary dataset not found. Looking for any CSV...")
    csv_files = [f for f in uploaded.keys() if f.endswith('.csv')]
    if csv_files:
        df_binary, acc_binary = process_dataset(csv_files[0], is_binary=True)
        results['binary'] = {'accuracy': acc_binary, 'df': df_binary}

# Process MULTI-CLASS dataset
multi_files = [f for f in uploaded.keys() if 'cleaned_file2' in f or any(x in f.lower() for x in ['multi', 'cleaned'])]
if multi_files:
    multi_file = multi_files[0]
    df_multi, acc_multi = process_dataset(multi_file, is_binary=False)
    results['multi'] = {'accuracy': acc_multi, 'df': df_multi}

# ===============================================
# STEP 8: SAVE RESULTS
# ===============================================
print("\n💾 Saving results...")

if 'binary' in results:
    binary_file_out = "airavata_binary_results.xlsx"
    results['binary']['df'].to_excel(binary_file_out, index=False)
    print(f"✅ Binary results saved: {binary_file_out}")

if 'multi' in results:
    multi_file_out = "airavata_multiclass_results.xlsx"
    results['multi']['df'].to_excel(multi_file_out, index=False)
    print(f"✅ Multi-class results saved: {multi_file_out}")

# Summary
print("\n" + "="*50)
print("🎯 FINAL RESULTS SUMMARY")
print("="*50)
for dataset_type, res in results.items():
    print(f"{dataset_type.upper()}: {res['accuracy']:.3f} accuracy")

# Download results
for file_out in [f for f in globals() if f.endswith('_results.xlsx')]:
    if file_out in locals():
        files.download(locals()[file_out])

print("\n✅ All done! Check downloaded Excel files for detailed results.")


Loading Airavata model... (this takes ~5-10 mins)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Airavata model loaded successfully!

📁 Upload your datasets:
- Binary: 'final_cleaned_data.csv' (columns: Text, Label, Word_Count)
- Multi-class: 'cleaned_file2 (1).csv' or similar


In [ ]:
def fake_news_prompt(text):
    # We provide examples (Few-Shot) to help the model understand Hindi context
    return f"""निर्देश: नीचे दिए गए समाचार को 'FAKE' या 'REAL' के रूप में वर्गीकृत करें। केवल एक शब्द में उत्तर दें।

उदाहरण 1:
समाचार: दिल्ली में कल से पेट्रोल 10 रुपये सस्ता होगा।
उत्तर: FAKE

उदाहरण 2:
समाचार: भारत ने जी20 शिखर सम्मेलन की मेजबानी की।
उत्तर: REAL

समाचार: {text}
उत्तर:"""

def predict_fake_news(text):
    prompt = fake_news_prompt(text)

    # We use a small max_new_tokens because we only want one word
    output = pipe(
        prompt,
        max_new_tokens=10,
        do_sample=False,
        truncation=True
    )[0]["generated_text"]

    # Extract the part after the last "उत्तर:"
    raw_answer = output.split("उत्तर:")[-1].strip().upper()

    # Robust check: if the word FAKE is anywhere in the response, count it
    if "FAKE" in raw_answer:
        return "FAKE"
    elif "REAL" in raw_answer:
        return "REAL"
    else:
        return "UNKNOWN" # To catch errors

In [ ]:
import pandas as pd

# Load your data (assuming df is already loaded from your Excel/CSV)
# Standardize names
df.columns = df.columns.str.lower()
if 'post' in df.columns:
    df = df.rename(columns={"post": "text", "labels set": "label"})

# Take a sample
df_small = df.sample(100, random_state=42).copy()

print("Processing 100 samples... this may take a few minutes.")
df_small["prediction_raw"] = df_small["text"].apply(predict_fake_news)

# Map predictions to match the integer labels (0 and 1)
mapping = {"FAKE": 0, "REAL": 1}
df_small["prediction"] = df_small["prediction_raw"].map(mapping)

# Ensure y_true is integer
y_true = df_small["label"].astype(int)
y_pred = df_small["prediction"].fillna(0).astype(int) # Fill unknown as 0 (Fake)

# Final Results
print("\n" + "="*30)
print(f"FINAL ACCURACY: {accuracy_score(y_true, y_pred):.2%}")
print("="*30)
print("\nFull Report:\n", classification_report(y_true, y_pred))